followed this post 'https://wikidocs.net/157265'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import seaborn as sns
from torch.utils.data import random_split, DataLoader, Dataset

In [2]:
import torch
from torch.nn.functional import one_hot
from torchvision.transforms import ToTensor

In [3]:
from torch import nn

In [4]:
class Iris_CustomDataset(Dataset):
    def __init__(self):
        self.data = pd.read_csv('Iris.csv', encoding='latin1')
        self.data['Species'] = self.data['Species'].replace(['Iris-virginica','Iris-setosa','Iris-versicolor'],[0,1,2])
        self.x_data = self.data[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values
        self.x_data = torch.from_numpy(self.x_data).float()
        
        self.y_data = self.data['Species'].values
        self.y_data = torch.from_numpy(self.y_data)
        self.y_data = one_hot(self.y_data)
        self.y_data = self.y_data.float()
        self.l = len(self.data)
        
    def __getitem__(self, index):
        return torch.FloatTensor(self.x_data[index]), torch.FloatTensor(self.y_data[index])
        
    def __len__(self):
        return self.l

In [5]:
dataset = Iris_CustomDataset()

In [ ]:
train_data

In [ ]:
len(dataset)

In [6]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size

In [7]:
train_data, test_data = random_split(dataset, [train_size, test_size])

In [ ]:
len(train_data)

In [8]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc_layer = nn.Sequential(
            nn.Linear(4, 3),
            nn.Softmax()
        )
    def forward(self, x):
        return self.fc_layer(x)

In [9]:
device = ('cuda')

In [10]:
model = Network().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters())

In [11]:
train_dataloader = DataLoader(dataset=train_data, batch_size=10)
test_dataloader = DataLoader(dataset=test_data, batch_size=10)

In [ ]:
next(model.parameters()).is_cuda

In [12]:
size = len(train_dataloader.dataset)
for i in range(40):
    for batch, (x, y) in enumerate(train_dataloader):
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if i % 40 == 0:
        print(f'[{i}  /  200]')

/home/jetson/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[0  /  200]


In [14]:
pred

tensor([[3.4175e-04, 7.6823e-01, 2.3143e-01],
        [2.6161e-04, 8.1255e-01, 1.8719e-01],
        [6.3202e-04, 7.7293e-01, 2.2644e-01],
        [1.1790e-04, 2.3154e-01, 7.6834e-01],
        [7.1900e-04, 7.4064e-01, 2.5864e-01],
        [2.7313e-04, 7.9909e-01, 2.0063e-01],
        [6.7986e-04, 7.6645e-01, 2.3287e-01],
        [7.6795e-06, 1.1679e-01, 8.8320e-01],
        [8.0672e-05, 2.4459e-01, 7.5533e-01],
        [8.6330e-05, 2.4965e-01, 7.5026e-01]], device='cuda:0')

In [13]:
size = len(test_dataloader.dataset)
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for x, y in test_dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        test_loss += loss_fn(pred, y)
        correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
        
print(f'final test_loss : {test_loss}')
test_loss /= 200
print(f'final correct : {correct}')
correct /= size

print(f"Test Error: \n Accuracy : {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


/home/jetson/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


final test_loss : 2.701230764389038
final correct : 23.0
Test Error: 
 Accuracy : 76.7%, Avg loss: 0.013506



In [22]:
pred.argmax(1)

tensor([0, 0, 1, 1, 1, 0, 1, 1, 2, 2], device='cuda:0')

In [23]:
y

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.]], device='cuda:0')